In [0]:
%sql
 CREATE OR REPLACE FUNCTION explain_ufo(description STRING, location STRING, datetime STRING)
  RETURNS STRING
  RETURN ai_query(
    'databricks-llama-4-maverick',
    "can you explain what would be the best explanation for this ufo sightseeing: " || description);

-- GRANT EXECUTE ON correct_grammar TO ds;

SELECT explain_ufo("silent red /orange mass of energy floated by three of us in western North Carolina in the 60s", "", "");



In [0]:
%sql
WITH text AS (
  SELECT "silent red /orange mass of energy floated by three of us in western North Carolina in the 60s" AS description
)
SELECT *,
  ai_query(
    'komae-openai-gpt-4-5',
    "can you explain what would be the best explanation for this ufo sightseeing: " || description
  )
FROM text;

In [0]:
%sql
WITH text AS (
  SELECT "silent red /orange mass of energy floated by three of us in western North Carolina in the 60s" AS description
)
SELECT *,
  ai_query(
    'databricks-llama-4-maverick',
    "can you explain what would be the best explanation for this ufo sightseeing: " || description
  )
FROM text;

In [0]:
%sql
WITH text AS (
  SELECT "silent red /orange mass of energy floated by three of us in western North Carolina in the 60s" AS description
)
SELECT *,
  ai_query(
    'databricks-claude-3-7-sonnet',
    "can you explain what would be the best explanation for this ufo sightseeing: " || description
  )
FROM text;

In [0]:
# databricks-llama-4-maverick
# sonnet
# openai
# databricks-claude-3-7-sonnet
